In [1]:
import pandas as pd

df = pd.read_excel('data/data_5710_20220716.xlsx', index_col=0)
df

c:\Users\sms20\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
일자,,,,,,,,,,
2022/07/15,70100.0,-2000.0,-2.77,72100.0,72200.0,69400.0,1605289.0,1.127915e+11,3.119088e+13,444948291.0
2022/07/14,72100.0,200.0,0.28,71000.0,73400.0,70400.0,1597400.0,1.147697e+11,3.208077e+13,444948291.0
2022/07/13,71900.0,1600.0,2.28,70100.0,72200.0,70100.0,1156844.0,8.267943e+10,3.199178e+13,444948291.0
2022/07/12,70300.0,-1000.0,-1.40,70700.0,71000.0,69400.0,1082539.0,7.575270e+10,3.127986e+13,444948291.0
2022/07/11,71300.0,-500.0,-0.70,71900.0,72800.0,71100.0,979002.0,7.039877e+10,3.172481e+13,444948291.0
...,...,...,...,...,...,...,...,...,...,...
2021/07/21,145000.0,-7000.0,-4.61,153000.0,155500.0,144500.0,10409836.0,1.533955e+12,6.444673e+13,444460230.0
2021/07/20,152000.0,-2000.0,-1.30,151500.0,153000.0,149000.0,6652856.0,1.003229e+12,6.755795e+13,444460230.0
2021/07/19,154000.0,-3000.0,-1.91,155500.0,155500.0,154000.0,3475792.0,5.367092e+11,6.844688e+13,444460230.0
